In [1]:
# !pip install scipy
# !pip install tenacity
# !pip install tiktoken
# !pip install termcolor 
# !pip install openai
# !pip install requests

In [2]:
import json
import openai
import requests
from tenacity import retry, wait_random_exponential, stop_after_attempt
from termcolor import colored
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()
GPT_MODEL = "gpt-3.5-turbo-0613"

In [7]:
openai.api_key = os.getenv("OPENAI_API_KEY")

In [8]:
@retry(wait=wait_random_exponential(min=1, max=40), stop=stop_after_attempt(3))
def chat_completion_request(messages, functions=None, function_call=None, model=GPT_MODEL):
    headers = {
        "Content-Type": "application/json",
        "Authorization": "Bearer " + openai.api_key,
    }
    json_data = {"model": model, "messages": messages}
    if functions is not None:
        json_data.update({"functions": functions})
    if function_call is not None:
        json_data.update({"function_call": function_call})
    try:
        response = requests.post(
            "https://api.openai.com/v1/chat/completions",
            headers=headers,
            json=json_data,
        )
        return response
    except Exception as e:
        print("Unable to generate ChatCompletion response")
        print(f"Exception: {e}")
        return e

In [9]:
def pretty_print_conversation(messages):
    role_to_color = {
        "system": "red",
        "user": "green",
        "assistant": "blue",
        "function": "magenta",
    }
    formatted_messages = []
    for message in messages:
        if message["role"] == "system":
            formatted_messages.append(f"system: {message['content']}\n")
        elif message["role"] == "user":
            formatted_messages.append(f"user: {message['content']}\n")
        elif message["role"] == "assistant" and message.get("function_call"):
            formatted_messages.append(f"assistant: {message['function_call']}\n")
        elif message["role"] == "assistant" and not message.get("function_call"):
            formatted_messages.append(f"assistant: {message['content']}\n")
        elif message["role"] == "function":
            formatted_messages.append(f"function ({message['name']}): {message['content']}\n")
    for formatted_message in formatted_messages:
        print(
            colored(
                formatted_message,
                role_to_color[messages[formatted_messages.index(formatted_message)]["role"]],
            )
        )

In [39]:
functions = [
    {
        "name": "get_recipe",
        "description": "Get the recipe",
        "parameters": {
            "type": "object",
            "properties": {
                "ingredients": {
                    "type": "string",
                    "description": "Get all the ingredients in the recipe, e.g. flour, egg, milk,...",
                },
                # "measurement": {
                #     "type": "string",
                #     "description": "The measurement or quantity of the ingredient, e.g. 2 cups",
                # },
            },
            "required": ["ingredient"],
        },
    }
]

In [40]:
messages = []
messages.append({"role": "system", "content": "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."})
messages.append({"role": "user", "content": "I have 2 cups of flour"})
chat_response = chat_completion_request(
    messages, functions=functions
)
assistant_message = chat_response.json()["choices"][0]["message"]
messages.append(assistant_message)
assistant_message

{'role': 'assistant',
 'content': None,
 'function_call': {'name': 'get_recipe',
  'arguments': '{\n  "ingredients": "flour"\n}'}}

In [41]:
messages.append({"role": "user", "content": """
3 heaping tablespoons of peanut butter
3 tablespoons of sambal sauce
3 tablespoons of soy sauce
I tablespoons of honey
1 tablespoons of sesame oil
The juice of 1 lime
1 garlic, grated
1 small knob of ginger, grated
 cup of water
1 cup of mushrooms
1/2 cup of cabbage leaves
2 servings of soba noodles
Chopped scallion
Chopped cilantro
"""})
chat_response = chat_completion_request(
    messages, functions=functions, function_call={"name": "get_recipe"}
)
assistant_message = chat_response.json()["choices"][0]["message"]
messages.append(assistant_message)
assistant_message

{'role': 'assistant',
 'content': None,
 'function_call': {'name': 'get_recipe',
  'arguments': '{\n  "ingredients": "peanut butter, sambal sauce, soy sauce, honey, sesame oil, lime, garlic, ginger, water, mushrooms, cabbage leaves, soba noodles, scallion, cilantro"\n}'}}

In [54]:
data = assistant_message.get("function_call").get("arguments")

In [55]:
parsed_data = json.loads(data)
ingredients = parsed_data['ingredients']
ingredient_list = ingredients.split(', ')

In [56]:
ingredient_list

['peanut butter',
 'sambal sauce',
 'soy sauce',
 'honey',
 'sesame oil',
 'lime',
 'garlic',
 'ginger',
 'water',
 'mushrooms',
 'cabbage leaves',
 'soba noodles',
 'scallion',
 'cilantro']